## EM算法
EM算法指的是最大期望算法(Expectation Maximization Algorithm),是一种迭代算法，用于含有隐变量的概率参数模型的最大似然或极大后验概率估计。
### 1.Jensen不等式
#### 1.1 凹凸函数
定义：如果函数总是位于任何一条弦的下方，则该函数是凸的；如果函数总是位于任何一条弦的上方，则函数是凹的。  
用数学语言表述如下：  
如果函数f在某个区间上存在非负(正)的二阶导数，即$f''(x)\ge 0$，则f为该区间的凸函数。当x是向量时，如果其Hessian矩阵H是半正定的，那么f是凸函数。如果$f''(x)\ge 0$或者$H>0$,那么f是严格凸函数。  
  
凸函数的例子有$x^2,|x|,e^x,x \log x(x\ge 0)$;凹函数的例子有log x,$\sqrt{x}(x\ge 0)$等。
#### 1.2 Jensen不等式：
如果f是凸函数，X是随机变量，那么
$$E[f(X)]\ge f(E[X])$$
特别地，如果f是严格凸的，等式成立当且仅当P(X=E[X])=1,即X是常量。   
Jensen不等式应用于凹函数时，不等号方向反向。

### 2.EM算法
给定训练样本为${\{x_1,x_2,\cdots,x_n}\}$,样例之间相互独立，我们想找到每个样例的隐含类别z，能使得p(x,z)最大。(这里$z_i=j$可以看成样本$x_i$被划分成j类) 记模型参数为$\theta$。  
训练样本的对数极大似然函数如下：  
\begin{align}
l(\theta)&=\sum_{i=1}^n\log p(x_i;\theta)\\
&=\sum_{i=1}^n\log\sum_{z_i}p(x_i,z_i;\theta)
\end{align}
第一步是对极大似然函数取对数，第二步是对每个样例的每个可能的类别j求联合分布概率和，但是直接求$\theta$一般比较困难，因为有隐变量z的存在，但是一般确定了z之后，求解就很容易了。  
### 2.1思想
EM是一种解决存在隐含变量优化问题的有效方法。既然不能直接最大化$l(\theta)$,我们可以不断建立$l(\theta)$的下界(E步),然后不断优化下界(M步)。

### 2.2分析
事实上，$z_i$也是个随机变量，对于每一个$x_i$,都有多种分类的情况。设第i个样本$x_i$在z上的概率分布为$Q_i(z_i)$,即$Q_i(z_i=j)$表示$x_i$被划分到类j的概率，因此有$\sum\limits_{z_i}Q_i(z_i)=1$。   
(如果z是连续性的，那么$Q_i(z_i)$是概率密度函数，需要将求和符号换做积分符号.)  
比如要将班上的同学聚类，假设隐变量z是身高，那么就是连续的高斯分布。如果隐变量是男女，那么就是伯努利分布了。   
进行变换得到以下公式：  
\begin{align}
\sum_{i=1}^n\log p(x_i;\theta)&=\sum_{i=1}^n\log\sum_{z_i}p(x_i,z_i;\theta)\qquad(1)\\
&=\sum_{i=1}^n\log\sum\limits_{z_i}Q_i(z_i)\dfrac{p(x_i,z_i;\theta)}{Q_i(z_i)}\quad(2)\\
&\ge \sum_{i=1}^n\sum\limits_{z_i}Q_i(z_i)\log\dfrac{p(x_i,z_i;\theta)}{Q_i(z_i)}\quad(3)
\end{align}  

这里，由(1)到(2)式比较直接，分子分母同乘以一个相等的；(2)到(3)利用了Jensen不等式，log(x)是凹函数，这里x=$\dfrac{p(x_i,z_i;\theta)}{Q_i(z_i)}$.

这个过程可以看作是对$l(\theta)$求了下界。假设$\theta$已经给定，那么$l(\theta)$的值就取决于$Q_i(z_i)$和$p(x_i,z_i;\theta)$了。我们通过调整这两个概率使下界不断上升，逼近$l(\theta)$的值。 
  
  
由Jensen不等式可知，要想等式成立，随机变量x=$\dfrac{p(x_i,z_i;\theta)}{Q_i(z_i)}$应为常数。故我们有：  
$$\dfrac{p(x_i,z_i;\theta)}{Q_i(z_i)}=c,\qquad c为常数$$

已知$\sum\limits_{z_i}Q_i(z_i)=1$，我们对上式进行变换并关于$z_i$求和得：
$$\sum\limits_{z_i}p(x_i,z_i;\theta)=c$$
代回原式解得：
\begin{align}
Q_i(z_i)&=\dfrac{p(x_i,z_i;\theta)}{\sum\limits_{z_i}p(x_i,z_i;\theta)}\\
&=\dfrac{p(x_i,z_i;\theta)}{p(x_i;\theta)}\\
&=p(z_i|x_i;\theta)
\end{align}

至此，我们推出了在固定$\theta$之后，$Q_i(z_i)$的计算公式就是后验概率，解决了$Q_i(z_i)$的选择问题，建立了$l(\theta)$的下界。接下来的M步，就是在给定$Q_i(z_i)$后，调整$\theta$,去极大化$l(\theta)$的下界。

### 2.3总结
一般的EM算法步骤如下：  
循环重复直至收敛{  
(E步)对于每一个i，计算  
$$Q_i(z_i):=p(z_i|x_i;\theta)$$
也等价于最大化$$E_z[\log\dfrac{p(x_i,z_i;\theta)}{Q_i(z_i)}]$$
(M步)计算  
$$\theta:=arg\max\limits_{\theta}\sum_{i=1}^n\sum\limits_{z_i}Q_i(z_i)\log\dfrac{p(x_i,z_i;\theta)}{Q_i(z_i)}$$
}

## 3.EM收敛性证明
假定$\theta^{(t)}$和$\theta^{(t+1)}$是EM第t次和第t+1次迭代后的结果。如果我们证明了$l(\theta^{(t)})\le l(\theta^{(t+1)})$,也就是说极大似然估计单调增加，那么我们最终会到达极大似然估计的最大值。  
证明如下：  
选定$\theta^{(t)}$后，我们得到E步
$$Q_i^{(t)}(z_i):=p(z_i|x_i;\theta^{(t)})$$
这一步保证了在给定$\theta^{(t)}$时，Jensen不等式成立，也就是
$$l(\theta^{(t)})=\sum_{i=1}^n\sum\limits_{z_i}Q_i^{(t)}(z_i)\log\dfrac{p(x_i,z_i;\theta^{(t)})}{Q_i^{(t)}(z_i)}$$
然后进行M步，固定$Q_i^{(t)}(z_i)$,并将$\theta^{(t)}$视作变量，对上面的$l(\theta^{(t)})$求导后，得到$\theta^{(t+1)}$,这样经过一些推导会有以下狮子成立：  
